In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem1234561"}
# %mpy -s {"/dev/cu.usbmodem101"}
%mpy -s {"/dev/cu.usbmodem2101"}

In [ ]:
!mpremote reset

In [ ]:
# %%micropython
import network
import time
import socket
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.protocols.socket_protocol import SocketProtocol
from mp_libs.protocols.serial_protocols import SerialProtocol


wifi = WifiProtocol(ssid="D&A_Home", password="welivewiththeTrees7")
socket_proto = SocketProtocol(True, ip="192.168.0.9")
serial_proto = SerialProtocol(transport=socket_proto, mtu_size_bytes=256)

wifi.connect()
print(f"client info: {wifi._sta.ifconfig()}")

print("Connecting to server...")
serial_proto.connect()

counter = 0
while True:
    rxed_data = []
    serial_proto.receive(rxed_data, timeout_msec=500)

    if rxed_data:
        print(f"Rx'ed ACK: {rxed_data[0]}")

        if rxed_data[0] == f"ACK - {counter}":
            counter += 1
        else:
            print("ABORT! Missed ack counter")
            print(f"Expected: {counter}")
            print(f"Received: {rxed_data[0]}")
            raise Exception("Done")
    else:
        print("Sending data...")
        serial_proto.send("Hello world")

print("Disconnecting...")
serial_proto.disconnect()
wifi.disconnect()